In [4]:
import numpy as np
import glob
import time

#It's kk to import whatever you want from the local util module if you would like:
#from util.X import ... 
%pylab inline 
    # Load Image
data_dir = '../data/'

easy_images = glob.glob(data_dir + 'easy/*/*')
medium_images = glob.glob(data_dir + 'medium_1/*/*')
medium_images.extend(glob.glob(data_dir + 'medium_2/*/*'))
hard_images = glob.glob(data_dir + 'hard/*/*')
print(hard_images)
tic = time.time()
im = imread('../data/easy/brick/brick_1.jpg')
# im = imread('../data/easy/ball/ball_2.jpg')
    
    
    
def convert_to_grayscale(im):
    '''
    Convert color image to grayscale.
    Args: im = (nxmx3) floating point color image scaled between 0 and 1
    Returns: (nxm) floating point grayscale image scaled between 0 and 1
    np.all(im == [255, 0, 0], axis = -1)
    '''
    return np.mean(im, axis = 2)

def roberts_cross(x):
    '''Compute Robert's Cross of input image x.
       Args: x (nxm) grayscale floating point image
       Returns: (n-1) x (m-1) edge image.'''
    
    #Our output image will be one pixel smaller than our image
    edges = np.zeros((x.shape[0]-1,x.shape[1]-1)) 

    for i in range(x.shape[0]-1):
        for j in range(x.shape[1]-1):
            #Grab Appropriate (2x2) image patch
            image_patch = x[i:i+2, j:j+2]
            # Compute Robert's Cross for image patch
            edges[i, j] = np.sqrt((image_patch[0,0] - image_patch[1, 1])**2 + 
                                   (image_patch[1, 0] - image_patch[0, 1])**2)
            
    return edges
def filter_2d(im, kernel):
    '''
    Filter an image by taking the dot product of each 
    image neighborhood with the kernel matrix.
    Args:
    im = (H x W) grayscale floating point image
    kernel = (M x N) matrix, smaller than im
    Returns: 
    (H-M+1 x W-N+1) filtered image.
    '''

    M, N = kernel.shape
    H, W = im.shape
    filtered_image = np.zeros((H-M+1, W-N+1), dtype = 'float64')
    
    for i in range(filtered_image.shape[0]):
        for j in range(filtered_image.shape[1]):
            image_patch = im[i:i+M, j:j+N]
            filtered_image[i, j] = np.sum(np.multiply(image_patch, kernel))
            
    return filtered_image
def make_gaussian_kernel(size, sigma):
    '''
    Create a gaussian kernel of size x size. 
    Args: 
    size = must be an odd positive number
    sigma = standard deviation of gaussian in pixels
    Returns: A floating point (size x size) guassian kernel 
    ''' 
    #Make kernel of zeros:
    kernel = np.zeros((size, size))
    
    #Handle sigma = 0 case (will result in dividing by zero below if unchecked)
    if sigma == 0:
        return kernel 
    
    #Helpful for indexing:
    k = int((size-1)/2)
    
    for i in range(size):
        for j in range(size):
            kernel[i, j] = (1/(2*np.pi*sigma**2))*exp(-((i-k)**2 + (j-k)**2)/(2*sigma**2))
            
    return kernel


# by Robert's cross

def classify_Roberts_cross(im):
    '''
    Example submission for coding challenge. 
    
    Args: im (nxmx3) unsigned 8-bit color image 
    Returns: One of three strings: 'brick', 'ball', or 'cylinder'
    
    '''
    #Let's guess randomly! Maybe we'll get lucky.
#     labels = ['brick', 'ball', 'cylinder']
#     random_integer = np.random.randint(low = 0, high = 3)
    
#     return labels[random_integer]

# -------------------------------------------------------
    im_scaled= im/255.
    gray=convert_to_grayscale(im_scaled)
    kernel = (1/9)*np.ones((3,3))
    gaussian_kernel = make_gaussian_kernel(size=11,sigma=2)
    im_filtered = filter_2d(gray,gaussian_kernel)
#     y=gray
    edges = roberts_cross(im_filtered)
    fig = figure(0, (24,12))
    fig.add_subplot(1,2,1)
    imshow(gray, cmap = 'gray')

    fig.add_subplot(1,2,2)
    imshow(edges)
#     colorbar()
    title("Edges computed with Robert's Cross")

def classify_Sobel_Feldman(im):
    Kx = np.array([[1, 0, -1],
               [2, 0, -2],
               [1, 0, -1]])

    Ky = np.array([[1, 2, 1],
               [0, 0, 0],
               [-1, -2, -1]])
    gray = convert_to_grayscale(im/255.)
    Gx = filter_2d(gray, Kx)
    Gy = filter_2d(gray, Ky)

    G_magnitude = np.sqrt(Gx**2+Gy**2)
    G_direction = np.arctan2(Gy, Gx)
    thresh = .55
    edges_and_angles = np.zeros(G_magnitude.shape)*np.NaN #Create empty array of NaNs
    #Replace pixels with gradient estimates above thresh with the direction of the gradient estimate:
    edges_and_angles[G_magnitude>thresh] = G_direction[G_magnitude>thresh] 
    fig = figure(0, (16,12))
#     imshow(edges_and_angles)
#     colorbar();
    cmap = matplotlib.cm.get_cmap('viridis')

    r = (np.max(G_direction) - np.min(G_direction)) #range
    direction_scaled = (G_direction - np.min(G_direction))/r

    rgba = cmap(direction_scaled) 
    rgb = rgba[:,:,:-1]

    grad_rep = np.repeat(np.expand_dims(G_magnitude, 2), 3, axis = 2)
    scaled_im = np.multiply(rgb, grad_rep)

#     fig = figure(0, (16,12))
#     imshow(scaled_im)
    hist(edges_and_angles.ravel(), 100); grid(1)
    print(edges_and_angles.shape)
#     print("Peaks "+str(find_peaks(edges_and_angles)))
    toc = time.time()
#     print(toc-tic:0.4f)
#     print(str(toc-tic)+"seconds")
    print(edges_and_angles)

def peakDetection(array):
    import math;
    rows = array.shape[0]
    cols = array.shape[1]
#     temp = [cols]
    temp =np.zeros(cols)
    for x in range(0, cols - 1):
        for y in range(0, rows -1):
            temp[x]=max(temp[x],array[x][y])
#     print(temp)  
    temp2= temp.nonzero()
    
def detect_peaks(x, mph=None, mpd=1, threshold=0, edge='rising',
                 kpsh=False, valley=False, ax=None):
    x = np.atleast_1d(x).astype('float64')
    if x.size < 3:
        return np.array([], dtype=int)
    if valley:
        x = -x
    # find indexes of all peaks
    dx = x[1:] - x[:-1]
    # handle NaN's
    indnan = np.where(np.isnan(x))[0]
    if indnan.size:
        x[indnan] = np.inf
        dx[np.where(np.isnan(dx))[0]] = np.inf
    ine, ire, ife = np.array([[], [], []], dtype=int)
    if not edge:
        ine = np.where((np.hstack((dx, 0)) < 0) & (np.hstack((0, dx)) > 0))[0]
    else:
        if edge.lower() in ['rising', 'both']:
            ire = np.where((np.hstack((dx, 0)) <= 0) & (np.hstack((0, dx)) > 0))[0]
        if edge.lower() in ['falling', 'both']:
            ife = np.where((np.hstack((dx, 0)) < 0) & (np.hstack((0, dx)) >= 0))[0]
    ind = np.unique(np.hstack((ine, ire, ife)))
    # handle NaN's
    if ind.size and indnan.size:
        # NaN's and values close to NaN's cannot be peaks
        ind = ind[np.in1d(ind, np.unique(np.hstack((indnan, indnan-1, indnan+1))), invert=True)]
    # first and last values of x cannot be peaks
    if ind.size and ind[0] == 0:
        ind = ind[1:]
    if ind.size and ind[-1] == x.size-1:
        ind = ind[:-1]
    # remove peaks < minimum peak height
    if ind.size and mph is not None:
        ind = ind[x[ind] >= mph]
    # remove peaks - neighbors < threshold
    if ind.size and threshold > 0:
        dx = np.min(np.vstack([x[ind]-x[ind-1], x[ind]-x[ind+1]]), axis=0)
        ind = np.delete(ind, np.where(dx < threshold)[0])
    # detect small peaks closer than minimum peak distance
    if ind.size and mpd > 1:
        ind = ind[np.argsort(x[ind])][::-1]  # sort ind by peak height
        idel = np.zeros(ind.size, dtype=bool)
        for i in range(ind.size):
            if not idel[i]:
                # keep peaks with the same height if kpsh is True
                idel = idel | (ind >= ind[i] - mpd) & (ind <= ind[i] + mpd) \
                    & (x[ind[i]] > x[ind] if kpsh else True)
                idel[i] = 0  # Keep current peak
        # remove the small peaks and sort back the indexes by their occurrence
        ind = np.sort(ind[~idel])

    return ind
# def peakfinder(d):
#     dh = hist(d,plot=False)
#     ins = dh[["intensities"]]
#     nbins = length(ins)
#     ss = which(rank(ins)%in%seq(from=nbins-2,to=nbins)) ## pick the top 3 intensities
#     return dh[["mids"]][ss]
# def find_peaks(a):
#     x = np.array(a)
#     max = np.max(x)
#     lenght = len(a)
#     ret = []
#     for i in range(lenght):
#         ispeak = True
#         if i-1 > 0:
#             ispeak &= (x[i] > 1.8 * x[i-1])
#         if i+1 < lenght:
#             ispeak &= (x[i] > 1.8 * x[i+1])

#         ispeak &= (x[i] > 0.05 * max)
#         if ispeak:
#             ret.append(i)
#     return ret

def findMode(G):
    temp=G.argmax(0)
    m = mean(temp)
    item = temp>m
    temp1 = []
    from collections import Counter 
    for i in range(0,item.size,5):
        n_num = item[i:i+5] 
        n = len(n_num) 
        data = Counter(n_num) 
        get_mode = dict(data) 
        mode = [k for k, v in get_mode.items() if v == max(list(data.values()))] 
        temp1.append(mode[0])
    return temp1


def countEdges(arr):
    count=0
    for i in range(len(arr)):
        print(arr[i])
        if i==0:
            if arr[i]==True:
                count=count+1
                
        else:
            if arr[i-1]==False & arr[i]==True:
                count=count+1
            
    return count 
    
def classify_hough(im):
    gray = convert_to_grayscale(im/255.)
    Kx = np.array([[1, 0, -1],
               [2, 0, -2],
               [1, 0, -1]])

    Ky = np.array([[1, 2, 1],
               [0, 0, 0],
               [-1, -2, -1]])
    gray = convert_to_grayscale(im/255.)
    Gx = filter_2d(gray, Kx)
    Gy = filter_2d(gray, Ky)
    G_magnitude = np.sqrt(Gx**2+Gy**2)
    G_direction = np.arctan2(Gy, Gx)
    edges = G_magnitude > 0.25
    y_coords, x_coords = np.where(edges)
    fig = figure(0, (12,9))
    scatter(x_coords, y_coords, s = 5)
    grid(1)
    toc = time.time()
    print(str(toc-tic)+"seconds")
    
def findMean(G):
#     return mean(G)
    return np.percentile(G,78)
    
def findMode1(G):
    min_g = min(G)
#     print(" Min "+str(min_g))
    max_g = max(G)
#     print(" Max "+str(max_g))
    avg = min_g + max_g

    G=G-min_g
#     per50 = np.percentile(G,50,axis=0)
#     per50 = np.percentile(G,50)
#     m = mean(G)
#     print("Mean "+str(m))
    item = G>avg
    return item
    
def classify_Sobel_Feldman1(im, i, num_rows, num_cols):
    Kx = np.array([[1, 0, -1],
                   [2, 0, -2],
                   [1, 0, -1]])
    Ky = np.array([[1, 2, 1],
                   [0, 0, 0],
                   [-1, -2, -1]])
    gray = convert_to_grayscale(im/255.)
    Gx = filter_2d(gray, Kx)
    Gy = filter_2d(gray, Ky)
    G_magnitude = np.sqrt(Gx**2+Gy**2)
    G_direction = np.arctan2(Gy, Gx)
    thresh = findMean(G_magnitude)
#     print("Threshold :"+str(thresh))
#     direction_of_edges=G_direction[G_magnitude>thresh]
#     len(direction_of_edges)
    fig.add_subplot(num_rows, num_cols, (i*2)+1)
    edges_and_angles = np.zeros(G_magnitude.shape)*np.NaN #Create empty array of NaNs
#Replace pixels with gradient estimates above thresh with the direction of the gradient estimate:
    edges_and_angles[G_magnitude>thresh] = G_direction[G_magnitude>thresh]
    
    imshow(edges_and_angles)
#     print(edges_and_angles)
    title(medium_images[i]+" "+str(thresh))
    fig.add_subplot(num_rows, num_cols, (i*2)+2)
    edges_and_angles = edges_and_angles[~numpy.isnan(edges_and_angles)]
    plt.hist(edges_and_angles,60)  
    counts, bin_edges = np.histogram(edges_and_angles, bins=60)
#     print("count: "+str(counts))
#     print("Bin_edges"+str(bin_edges))
#     (n, bins, patches) = np.histogram(direction_of_edges,50)
#     print("Hist "+str(counts))
    counts.astype(int)
    
#     print("counts :"+str(counts))
#     boolArray = findMode1(counts)
#     np_array = np.array(boolArray)
#     peaks =np.count_nonzero(np_array[:-1] < np_array[1:])
#     print("Peaks "+str(peaks))
#     return peaks
    peak_h = detect_peaks(counts, mph=max(counts)*30/100, mpd=6,threshold=max(counts)*30/100)
    peak_w = detect_peaks(counts, mph=max(counts)*10/100, mpd=15)
    newPeaks = []
    newWidth = []
    for i in range(len(peak_h)):
        newPeaks.append(counts[peak_h[i]])
    for i in range(len(peak_w)):
        newWidth.append(counts[peak_w[i]])
        
#     peak_w = detect_peaks(counts, mph=max(counts)*10/100, mpd=7)
#     print("Peaks "+str(peaks))
    print(str(newPeaks)+" "+str(newWidth))
#     return [len(peak_h), len(peak_w)]
    
    return [len(newPeaks), len(newWidth)]
    
    

fig = figure(0, (28,170))
num_cols = 2
# num_rows = int(ceil(len(hard_images)/num_cols))
num_rows = int(ceil(len(medium_images)))
for i in range(len(medium_images)):
    im=imread(medium_images[i])
    peaks = classify_Sobel_Feldman1(im, i, num_rows, num_cols)
#     print(peaks)
    peak = peaks[0]
    width = peaks[1]
#     max_c = peaks[1]
#     min_c = peaks[2]


    if(peak>=4 and width >=3):
        print(medium_images[i]+': Brick')
    elif(peak<4 and width <3):
        print(medium_images[i]+': Ball')
    elif( width>3):
        print(medium_images[i]+': Cylinder')
          
#     if  peak<=3 and min_c>=max_c/3:
#         print(medium_images[i]+': Ball')
#     elif peak>=4 and min_c<max_c/3:
#         print(medium_images[i]+': Brick')
#     elif(peak>=4 and min_c>=max_c/3):
#         print(medium_images[i]+': Cylinder')
# im=imread(hard_images[0])
# classify_Sobel_Feldman1(im, 0, 1, 1)
toc = time.time()
print(str(toc-tic) + " Seconds") 

Populating the interactive namespace from numpy and matplotlib
[]


FileNotFoundError: [Errno 2] No such file or directory: '../data/easy/brick/brick_1.jpg'